In [ ]:
import numpy as np
from plotly import express as px

In [ ]:
eps = 1e-10
alpha = 0
N = 1000 # на сколько подотрезков бить [0, 1]

In [ ]:
def f1(x1, x2, p1, p2):
    return x2


def f2(x1, x2, p1, p2):
    return p2 + x1 * np.exp(alpha * x1)


def f3(x1, x2, p1, p2):
    return -p2 * (1 + alpha * x1) * np.exp(alpha * x1)


def f4(x1, x2, p1, p2):
    return -p1


def calc_k(x1, x2, p1, p2, k1=0, k2=0, k3=0, k4=0, h=0):
    X = np.array([x1, x2, p1, p2]) + 0.5 * h * np.array([k1, k2, k3, k4])
    return f1(*X), f2(*X), f3(*X), f4(*X)


def gamma(v, k1, k2, k3, k4, h):
    return v + h * (k1 + 2 * (k2 + k3) + k4) / 6


def gammas(vs, h):
    ks1 = calc_k(*vs)
    ks2 = calc_k(*vs, *ks1, h)
    ks3 = calc_k(*vs, *ks2, h)
    ks4 = calc_k(*vs, *ks3, h)
    return [gamma(v, ks1[i], ks2[i], ks3[i], ks4[i], h) for i, v in enumerate(vs)]

In [ ]:
def solve(a, b):
    t = np.linspace(0, 1, N)
    xs1 = np.zeros(t.shape)
    xs2 = np.zeros(t.shape)
    ps1 = np.zeros(t.shape)
    ps2 = np.zeros(t.shape)

    # Начальные условия
    xs1[0], xs2[0], ps1[0], ps2[0] = 0, 0, a, b
    h = t[1] - t[0]

    for i in range(1, len(t)):
        xs1[i], xs2[i], ps1[i], ps2[i] = gammas((xs1[i - 1], xs2[i - 1], ps1[i - 1], ps2[i - 1]), h)

    return xs1, xs2, ps1, ps2, t

In [ ]:
def err_x1(res):
    x1T = np.sinh(1)
    return res[0][-1] - x1T


def err_x2(res):
    x2T = np.exp(1)
    return res[1][-1] - x2T


def err_total(res):
    return abs(err_x1(res)) + abs(err_x2(res))


def calc_near_solutions(a, b, h):
    return solve(a, b), solve(a + h, b), solve(a, b + h)


def shooting(a0=1, b0=1, max_iter=100):
    a, b = a0, b0
    counter = 0	
    h = 1 / N
    sol_ab, sol_ahb, sol_abh = calc_near_solutions(a, b, h)
    err = err_total(sol_ab)
    try:
        while err > eps and counter < max_iter:
            phi1, phi1a, phi1b = err_x1(sol_ab), err_x1(sol_ahb), err_x1(sol_abh)
            phi2, phi2a, phi2b = err_x2(sol_ab), err_x2(sol_ahb), err_x2(sol_abh)
            W = np.array([
                [(phi1a - phi1) / h, (phi1b - phi1) / h],    #
                [(phi2a - phi2) / h, (phi2b - phi2) / h]
            ])
            da, db = np.dot(np.linalg.inv(W), [phi1, phi2])
            a, b = a - da, b - db
            sol_ab, sol_ahb, sol_abh = calc_near_solutions(a, b, h)
            err = err_total(sol_ab)
            counter += 1
    except Exception as ex:
        print('Shooting stop with exception:', ex)
        print('Output maybe incorrect')
    print('Final error:', err, 'Total iterations:', counter)

    return a, b


In [ ]:
def x1_teor(t):
    return t * np.sinh(t)


def x2_teor(t):
    return t * np.cosh(t) + np.sinh(t)

In [ ]:
a, b = shooting(a0=-10, b0=-1)
print('a=', a, 'b=', b)

res = solve(a, b)
x1, x2, p1, p2, t = res
u = p2

fig = px.line(width=720, height=480)
fig.add_scatter(x=t, y=x1, mode='lines', name='x')
# fig.add_scatter(x=t, y=x2, mode='lines', name='x2')
if alpha == 0:
    x1t = [x1_teor(t) for t in res[-1]]
    x2t = [x2_teor(t) for t in res[-1]]
    fig.add_scatter(x=t, y=x1t, mode='lines', name='x_teor')
    # fig.add_scatter(x=t, y=x2t, mode='lines', name='x2_teor')
fig.add_scatter(x=t, y=u, mode='lines', name='u')
fig.show()